In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import math
import random
import os

In [3]:
print("=" * 50)
print("예제 : 대한민국 구석구석 사이트 여행지 정보 수집하기")
print("=" * 50)
query_txt = input("1. 크롤링할 키워드는 무엇입니까? : ")
cnt = int(input("2. 크롤링할 건수는 몇건입니까? : "))
page_cnt = math.ceil(cnt / 10)
print("출력될 페이지수 : ", page_cnt)

예제 : 대한민국 구석구석 사이트 여행지 정보 수집하기
1. 크롤링할 키워드는 무엇입니까? : 제주도
2. 크롤링할 건수는 몇건입니까? : 50
출력될 페이지수 :  5


In [4]:
f_dir = input("3. 결과파일을 저장할 폴더명을 입력하세요 (예: C:\\LHR\\python\\notebook\\data\\) : ")
n = time.localtime() #현재시간
s = "{}-{}-{}-{}-{}-{}".format(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)
os.makedirs(f_dir + s +'-' + query_txt) #데이터에 디렉토리 생성

#폴더 경로와 키워드 이름을 넣은 파일 생성
ff_name = f_dir + s + '-' + query_txt + "\\" + s + '-' + query_txt + '.txt'
fc_name = f_dir + s + '-' + query_txt + "\\" + s + '-' + query_txt + '.csv'
fx_name = f_dir + s + '-' + query_txt + "\\" + s + '-' + query_txt + '.xlsx'

3. 결과파일을 저장할 폴더명을 입력하세요 (예: C:\LHR\python\notebook\data\) : C:\LHR\python\notebook\data\


In [5]:
s_time = time.time()

path = "C:/LHR/python/datadown/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get('https://korean.visitkorea.or.kr')
time.sleep(random.randrange(2,5)) # 2-5 초 사이에 랜덤으로 시간 선택

try :
    driver.find_element_by_xpath('//*[@id="chkForm01"]').click()
except :
    print("코로나 창이 없습니다")
    
element = driver.find_element_by_id("inp_search")
element.send_keys(query_txt)
element.send_keys("\n")

html = driver.page_source
soup = BeautifulSoup('html.parser')

print("=" * 50)
print("크롤링 할 키워드 : ", query_txt)
print("크롤링 할 총 페이지 번호 : ", page_cnt)
print("=" * 50)

크롤링 할 키워드 :  제주도
크롤링 할 총 페이지 번호 :  5


In [6]:
no2 = []
contents2 = []
no = 1

for x in range(1, page_cnt+1) :
    print("{}페이지 내용 수집을 시작합니다 =========================".format(x))
    
    #10개의 데이터 추출(광고가 있을 경우를 대비해 13으로 입력함)
    for i in range(1, 13):
        if no > cnt:
            break
        
        #각 게시글의 제목 클릭
        try:
            driver.find_element_by_xpath('''//*[@id="listBody"]/ul/li[{}]/div[2]/div[1]/a'''.format(i)).click()
        except:
            continue
            
        time.sleep(random.randrange(2,5))
        
        html = driver.page_source #상세페이지에 있는 값 읽어오기
        soup = BeautifulSoup(html, 'html.parser')
        
        try:
            #content_list = soup.find('div', 'wrap_contView')
            #con_1 = content_list.find('div', 'txt_p').get_text()
            con_1 = soup.find('div', 'txt_p').get_text()
        except:
            try:
                #con_0 = content_list.find('div', 'txt_p') #텍스트가 없는 경우 con_0에 담아서 다시 처리
                con_0 = soup.find('div', 'txt_p')
                con_1 = con_0.find('p').get_text #con_0 안에서 p태그를 찾아 텍스트로 변환
            except:
                driver.back( ) 
                continue #바로 올라가서 page_cnt가 증가함
        
        print(no, " : ", con_1)
        print("\n")
        
        #파일 저장
        f = open(ff_name, 'a', encoding='UTF-8')
        f.write(str(no) + ':' + str(con_1) + "\n")
        f.close()
        
        no2.append(no)
        contents2.append(con_1)
        
        driver.back() #뒤로가기 -> 목록 페이지로 이동
        time.sleep(2)
        
        no += 1
        
    if x > page_cnt + 1:
        break
        
    x += 1
    
    time.sleep(2)
    if(x % 5 == 1): #5랑 나누었을 때 1일때 다음을 누르도록
         driver.find_element_by_link_text("다음").click()
    else:
         driver.find_element_by_link_text("{}".format(x)).click() #다음페이지 클릭
        
    time.sleep(2)

1페이지 내용 수집을 시작합니다 =========================
1  :  제주하면 떠오르는 그것. ‘올레’라는 이름입니다. 올레라는 단어가 누군가에게는 제주도를 한 바퀴 도는 걷기 여행 코스일 수도 있고, 누군가에게는 소주의 이름일 수도 있습니다. ‘올레’는 큰길에서 집으로 들어가는 길목에 있는 작은 골목길을 뜻하는 제주도 방언입니다. 집 안쪽으로 들어서며 볼 수 있는 검은빛 현무암을 쌓아 만든 돌담길이 굽이굽이 이어지는 모습을 상상하셨나요? 맞습니다. 바로 그 골목길이 올레입니다.


2  :  전국 어디서나 만날 수 있는 흔한 음식이 닭이라지만, 제주에서는 이 닭요리도 맛깔스런 별미가 된다. 한라산 맑은 공기를 먹고 자란 청정 토종닭을 샤부샤부와 백숙, 칼국수 등으로 다양하게 맛볼 수 있는 곳. 제주도 토종닭 유통특구로 지정된 조천읍 교래리에서 여름철 잃었던 입맛을 되찾아보자. 한끼 식사에 몸도 마음도 든든해진다.


3  :  제주국제공항에서 차로 10분 거리, 제주시외버스터미널에서는 걸어서 5분 거리에 위치한 ‘제주R호텔’. 이동 시간을 줄이고 한 곳이라도 더 둘러보고자 한다면 이곳이 제격이다. 특히 자동차운전면허가 없는 ‘뚜벅이’나 제주도를 처음 방문하는 여행객에게 추천한다. 옛 여관을 레노베이션한 호텔은 밝은 오렌지색으로 새 옷을 입은 덕분에 분위기가 산뜻하다. 다양한 보드게임과 미니농구대 등을 갖춘 로비에 들어서니 마치 보드 카페에 온 듯하다. 프런트에 붙여 놓은 에티켓 안내중 객실에서 음식물을 섭취할 수 없다는 문구가 눈길을 끈다. “청결 때문이기도 하지만, 호텔 이용객을 위해서이기도 합니다. 로비에서 다른 여행자와 소통하길 바라는 마음이지요. 이곳은 제주도, 여행지잖아요.” 조창수 점장의 설명이다. 도란도란 사람들과 어울려 지내길 바라는, 제주도라서 가능한 제안과 실천이다.


4  :  아직 추위가 가시지 않은 겨울의 끝자락. 봄소식을 가장 먼저 알린다는 복수초가 이제 막 기지개를 켜는 참이다. 제주 한라생태숲에 피어나기 시작

41  :  문화체육관광부와 한국관광공사는 매월 각 지역별 여행하기 좋은 걷기여행길을 선정한다.  2월은 24절기 중 입춘이 있는 달로, 봄의 시작과 함께 길운을 바라는 입춘대길이라는 글귀를 볼 수 있는 계절이다. 2월은 우리나라에서 봄의 완연한 기운을 가장 빨리 느낄 수 있는 제주도의 걷기여행길 5곳을 선정하였다. 설 연휴를 맞아 가족, 연인, 친구와 함께 걷어보면 어떨까? 이달의 추천길로 선정된 길은 ‘두루누비(durunubi.kr)’에서 자세한 정보를 확인할 수 있다.


42  :  제주도와의 만남은 육지 이방인에겐 늘 설렘이다.이른 새벽에 비행기를 타고 도착해 늦은 밤 비행기로 되돌아오는 빡빡한 업무 일정 속에도 제주행 비행기의 탑승구를 오르는 발걸음은 소풍 길에 나서는 아이처럼 언제나 들떠 있다. 1주일이라는 비교적 긴 휴가기간을 할애해 리조트에서 뒹굴뒹굴 보낼 계획을 짤 때도 마찬가지. 그때까지 만나지 못한 신세계를 경험하게 될 것이란 막연한 기대감이 불쑥불쑥 기분 좋게 다가온다.


43  :  2010년 유네스코 세계지질공원으로 인증 받은 제주도는 섬 전체가 지질 과학관이다. 제주의 핵심 지질 명소 12곳에는 지질 자원을 테마로 한 이색 숙소와 먹거리가 있다. 일명 '지오하우스'와 '지오푸드'다. 핵심 지질 명소 가운데 제주도의 속살을 다채롭게 엿볼 수 있는 사계리를 찾았다. 80만 년이 층층이 쌓인 곳에서 지질을 테마로 걷고 먹고 여독을 풀었다.


44  :  지금 제주는 고사리가 한창이다. 한라산 자락을 따라 펼쳐진 오름이며 무성하게 우거진 수풀마다 온통 고사리 천국이다. 청정 자연에서 자라난 무공해 제주 고사리는 맛과 품질 면에서 단연 전국 최고로 꼽힌다. 제주 자연의 맛, 고사리 탐험에 나서보자.


45  :  날씨가 따뜻해지면서 가고 싶은 곳, 하고 싶은 것들이 점점 늘어간다. 오랫동안 보지 못한 친구도 만나고 싶고, 답답한 실내에서 벗어나 탁 트인 자연을 느끼고픈 마음. 이 모든 걸 아우르는 게 결국 '여행' 아닐까? 한 설문에 

In [7]:
korea = pd.DataFrame()
korea['번호']=no2
korea['내용']=contents2
korea.to_csv(fc_name, encoding="UTF-8-sig", index=False)
korea.to_excel(fx_name, index=False)

e_time = time.time()
t_time = e_time - s_time

print("\n")
print("=" * 50)
print("총 소요시간은 {}초 입니다 ".format(t_time,1))
print("파일 저장 완료: txt 파일명 : {} ".format(ff_name))
print("파일 저장 완료: csv 파일명 : {} ".format(fc_name))
print("파일 저장 완료: xls 파일명 : {} ".format(fx_name))
print("=" * 50)

driver.close()



총 소요시간은 432.2849454879761초 입니다 
파일 저장 완료: txt 파일명 : C:\LHR\python\notebook\data\2021-4-14-17-33-40-제주도\2021-4-14-17-33-40-제주도.txt 
파일 저장 완료: csv 파일명 : C:\LHR\python\notebook\data\2021-4-14-17-33-40-제주도\2021-4-14-17-33-40-제주도.csv 
파일 저장 완료: xls 파일명 : C:\LHR\python\notebook\data\2021-4-14-17-33-40-제주도\2021-4-14-17-33-40-제주도.xlsx 
